<a href="https://colab.research.google.com/github/godud2604/AI-bootcamp/blob/main/Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 보스턴 주택 가격 데이터 세트를 Peceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
* 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [1]:
import sklearn

print(sklearn.__version__)

1.0.2


In [12]:
!pip install scikit-learn==1.0.2

In [4]:
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

(506, 14)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Weight와 Bias의 Update 값을 계산하는 함수 생성.
* w1은 RM(방의 계수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* bias는 Bias
* N은 입력 데이터 건수
![](https://raw.githubusercontent.com/chulminkw/CNN_PG/main/utils/images/Weight_update.png)


In [5]:
# gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수.
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨.
# 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 loss로 반환.

def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
  # 데이터 건수
  N = len(target)

  # 예측 값
  predicted = w1 * rm + w2 * lstat + bias

  # 실제값과 예측값의 차이
  diff = target - predicted

  # bias를 array 기반으로 구하기 위해서 설정. (1을 N개만큼 생성)
  bias_factors = np.ones((N,))

  # weight와 bias를 얼마나 update할 것인지를 계산.
  w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
  w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
  bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))

  # Mean Squared Error값을 계산.
  mse_loss = np.mean(np.square(diff))

  # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환.
  return bias_update, w1_update, w2_update, mse_loss

### Gradient Descent 를 적용하는 함수 생성
* iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용.

In [6]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용.
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0] # 첫 번째 컬럼
    lstat = features[:, 1] # 두 번째 컬럼

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행.
    for i in range(iter_epochs):
        # weight/bias update 값 계산
        bias_update, w1_update, w2_update, loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        # weight/bias의 update 적용.
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', loss)

    return w1, w2, bias

### Gradient Descent 적용
* 신경망은 데이터를 정규화/표준화 작업을 미리 선행해 주어야 함.
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

print(scaled_features)

[[0.57750527 0.08967991]
 [0.5479977  0.2044702 ]
 [0.6943859  0.06346578]
 ...
 [0.65433991 0.10789183]
 [0.61946733 0.13107064]
 [0.47307913 0.16970199]]


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch: 2502 / 5000
w1: [23.9249342] w2: [-21.23815095] bias: [16.45256258] loss: 30.979973242429722
Epoch: 2503 / 5000
w1: [23.92651912] w2: [-21.24062335] bias: [16.45248123] loss: 30.979108879218803
Epoch: 2504 / 5000
w1: [23.92810265] w2: [-21.24309331] bias: [16.45239987] loss: 30.97824615628058
Epoch: 2505 / 5000
w1: [23.92968481] w2: [-21.24556086] bias: [16.45231849] loss: 30.97738507049069
Epoch: 2506 / 5000
w1: [23.93126558] w2: [-21.24802598] bias: [16.45223711] loss: 30.976525618730737
Epoch: 2507 / 5000
w1: [23.93284497] w2: [-21.25048868] bias: [16.45215571] loss: 30.975667797888264
Epoch: 2508 / 5000
w1: [23.93442299] w2: [-21.25294896] bias: [16.4520743] loss: 30.974811604856743
Epoch: 2509 / 5000
w1: [23.93599963] w2: [-21.25540682] bias: [16.45199288] loss: 30.973957036535577
Epoch: 2510 / 5000
w1: [23.93757489] w2: [-21.25786227] bias: [16.45191145] loss: 30.973104089830052
Epoch: 2511 / 5000
w1: [23.93914878] w2: [-21.26031531] bia

### 계산된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산.

In [11]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.935533
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.483093
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.545474
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.334142
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.516284
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,28.074722
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.342942
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,17.772340
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,8.129206
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.276548


### Keras를 이용하여 보스턴 주택가격 모델 학습 및 예측
* Dense Layer를 이용하여 퍼셉트론 구현. units는 1로 설정.

In [13]:
Dense??

In [12]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음.
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용.
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행.
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(scaled_features, bostonDF['PRICE'].values, epochs=1000)


Epoch 1/1000
16/16 [==============================] - 1s 2ms/step - loss: 542.5001 - mse: 542.5001
Epoch 2/1000
16/16 [==============================] - 0s 2ms/step - loss: 530.4066 - mse: 530.4066
Epoch 3/1000
16/16 [==============================] - 0s 2ms/step - loss: 518.4561 - mse: 518.4561
Epoch 4/1000
16/16 [==============================] - 0s 2ms/step - loss: 506.7931 - mse: 506.7931
Epoch 5/1000
16/16 [==============================] - 0s 2ms/step - loss: 495.2899 - mse: 495.2899
Epoch 6/1000
16/16 [==============================] - 0s 2ms/step - loss: 484.1107 - mse: 484.1107
Epoch 7/1000
16/16 [==============================] - 0s 2ms/step - loss: 473.1477 - mse: 473.1477
Epoch 8/1000
16/16 [==============================] - 0s 2ms/step - loss: 462.3571 - mse: 462.3571
Epoch 9/1000
16/16 [==============================] - 0s 2ms/step - loss: 451.8850 - mse: 451.8850
Epoch 10/1000
16/16 [==============================] - 0s 2ms/step - loss: 441.5587 - mse: 441.5587
Epoch 11/

### Keras로 학습된 모델을 이용하여 주택 가격 예측 수행.

In [14]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE'] = predicted
bostonDF.head(10)


16/16 [==============================] - 0s 1ms/step


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE,KERAS_PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.935533,28.973110
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.483093,25.497845
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.545474,32.630135
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.334142,32.408226
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.516284,31.594551
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,28.074722,28.101097
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.342942,21.318724
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,17.772340,17.741028
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,8.129206,8.028409
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.276548,18.238716


### Stochastic Gradient Descent와 Mini Batch Gradient Descent 구현
* SGD 는 전체 데이터에서 한건만 임의로 선택하여 Gradient Descent 로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용
* Mini Batch GD는 전체 데이터에서 Batch 건수만큼 데이터를 선택하여 Gradient Descent로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

### SGD 기반으로 Weight/Bias update 값 구하기

In [ ]:
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):

    # 데이터 건수
    N = target_sgd.shape[0]
    # 예측 값.
    predicted_sgd = w1 * rm_sgd + w2*lstat_sgd + bias
    # 실제값과 예측값의 차이
    diff_sgd = target_sgd - predicted_sgd
    # bias 를 array 기반으로 구하기 위해서 설정.
    bias_factors = np.ones((N,))

    # weight와 bias를 얼마나 update할 것인지를 계산.
    w1_update = -(2/N)*learning_rate*(np.dot(rm_sgd.T, diff_sgd))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_sgd.T, diff_sgd))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_sgd))

    # Mean Squared Error값을 계산.
    #mse_loss = np.mean(np.square(diff))

    # weight와 bias가 update되어야 할 값 반환
    return bias_update, w1_update, w2_update

### SGD 수행하기

In [ ]:
print(bostonDF['PRICE'].values.shape)
print(np.random.choice(bostonDF['PRICE'].values.shape[0], 1))
print(np.random.choice(506, 1))

In [ ]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용.
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]


    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행.
    for i in range(iter_epochs):
        # iteration 시마다 stochastic gradient descent 를 수행할 데이터를 한개만 추출. 추출할 데이터의 인덱스를 random.choice() 로 선택.
        stochastic_index = np.random.choice(target.shape[0], 1)
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        # SGD 기반으로 Weight/Bias의 Update를 구함.
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)

        # SGD로 구한 weight/bias의 update 적용.
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)

    return w1, w2, bias

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_SGD'] = predicted
bostonDF.head(10)

### iteration시마다 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [ ]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):

    # 데이터 건수
    N = target_batch.shape[0]
    # 예측 값.
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    # 실제값과 예측값의 차이
    diff_batch = target_batch - predicted_batch
    # bias 를 array 기반으로 구하기 위해서 설정.
    bias_factors = np.ones((N,))

    # weight와 bias를 얼마나 update할 것인지를 계산.
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))

    # Mean Squared Error값을 계산.
    #mse_loss = np.mean(np.square(diff))

    # weight와 bias가 update되어야 할 값 반환
    return bias_update, w1_update, w2_update

In [ ]:
batch_indexes = np.random.choice(506, 30)
print(batch_indexes)

bostonDF['RM'].values[batch_indexes]



In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음.
def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행.
    for i in range(iter_epochs):
        # batch_size 갯수만큼 데이터를 임의로 선택.
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        # Batch GD 기반으로 Weight/Bias의 Update를 구함.
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

        # Batch GD로 구한 weight/bias의 update 적용.
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)

    return w1, w2, bias

In [ ]:
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

### iteration 시에 순차적으로 일정한 batch 크기만큼의 데이터를 전체 학습데이터에 걸쳐서 가져오는 Mini-Batch GD 수행

In [ ]:
for batch_step in range(0, 506, 30):
    print(batch_step)

In [ ]:
bostonDF['PRICE'].values[480:510]

In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음.
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행.
    for i in range(iter_epochs):
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size):
            # batch_size만큼 순차적인 데이터를 가져옴.
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]

            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

            # Batch GD로 구한 weight/bias의 update 적용.
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update

            if verbose:
                print('Epoch:', i+1,'/', iter_epochs, 'batch step:', batch_step)
                # Loss는 전체 학습 데이터 기반으로 구해야 함.
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)

    return w1, w2, bias

In [ ]:
w1, w2, bias = batch_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

### Mini BATCH GD를 Keras로 수행
* Keras는 기본적으로 Mini Batch GD를 수행

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음.
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용.
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행.
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

# Keras는 반드시 Batch GD를 적용함. batch_size가 None이면 32를 할당.
model.fit(scaled_features, bostonDF['PRICE'].values, batch_size=30, epochs=1000)

In [ ]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)